# Compare the Afifi and Lakhnawi editions of the Fusus

We have two versions of the Fusus text, obtained in wildly different ways:

* `AF` the Afifi edition, obtained via the OCR pipeline
* `LK` the Lakhnawi edition,
   obtained by reverse engineering a textual PDF with unusual fonts and private use characters.
   
The results of both attempts have been cleaned and enriched by Cornelis van Lit,
through visual inspection and manipulation in Pandas.

In this notebook, we align the results obtained by Cornelis.
The essential result is an alignment table where the words of the `LK` are brought into
correspondence with the words in `AF`.

A candidate tool to perform sequence alignment with is
[Collatex](https://collatex.net) by Ronald Haentjes-Dekker.
We experimented with it, see the [collatexAfLk notebook](collatexAfLk.ipynb),
but it took a very long time to run.

However, the two editions are very similar, with very few transpositions but a lot of OCR errors.

We use Text-Fabric.

The cleaned `LK` and `AF` are present in the TF resources `fususl` and `fususa`.
We then have each word precisely numbered in both version, and we also have a latin
transcription of the words, which eases the visual comparison of similar words.

It is not only that the author of this notebook (Dirk Roorda) is not trained to read Arabic,
it is also that the Arabaic letters change shape depending on their position in the word,
which makes a task like this needlessly complicated.

In [1]:
%load_ext autoreload
%autoreload 2

# Edit distance

We need a measure to estimate how similar words are.
A good measure is the *edit distance*, i.e. the amount of edit operations to change
one word into another.
Since one of the editions is the result of OCR, we expect (many) OCR errors,
and for this the concept of edit distance is useful.

However, we also need the slightly more refined notion of *ratio*, which takes the lengths
of the words into account. It roughly corresponds to proportion of the common part of the words
in relation to the totality of the words.
In other words, and edit distance of 2 between words of length 10 tells you that the words are rather similar,
their *ratio* is high. But the same edit distance between words of length 3 means that the words are very
different, their *ratio* is low.

The concepts *ratio* and *edit distance* can be computed by the
[Levenshtein algorithm](https://en.wikipedia.org/wiki/Levenshtein_distance),
which is implemented in the Python module `Levenshtein`.

You may have to

```
pip3 install python-Levenshtein
```

In [2]:
from Levenshtein import distance, ratio

In [3]:
import collections
import os

from tf.app import use

In [4]:
BASE = os.path.expanduser("~/github/among/fusus")
VERSION = "0.7"
ALIGNMENT_DIR = f"{BASE}/ur/Fusus"
ALIGNMENT_PATH = f"{ALIGNMENT_DIR}/alignment{VERSION}.tsv"

!mkdir -p {ALIGNMENT_DIR}

# Load both editions

Normally, when we load a single data source in a notebook, we store the handle in a variable called
`A`, and we hoist additional variables such as `F` (access to feature data) into the global namespace.

But now we work with two datasources, so we store the handles in a dictionary `A`, with
a key `LK` for the Lakhnawi edition and a key `AF` for the Afifi edition.

We also make dictionaries for `F`, `maxSlot`, etc., keyed with the same keys, to store
edition dependent data and properties.

In that way we can systematically select our handles for the desired editions.

In [5]:
LK = "LK"
AF = "AF"

EDITIONS = {
    LK: "Lakhnawi",
    AF: "Afifi",
}

A = {}
F = {}
maxSlot = {}

In [6]:
for (acro, name) in EDITIONS.items():
    A[acro] = use(f"among/fusus/tf/{name}:clone", writing="ara", version=VERSION)
    F[acro] = A[acro].api.F
    maxSlot[acro] = F[acro].otype.maxSlot

This is Text-Fabric 9.1.3
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

27 features found and 0 ignored


This is Text-Fabric 9.1.3
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

17 features found and 0 ignored


Let's find out the max slot of both editions.

In [7]:
maxSlot

{'LK': 40379, 'AF': 40271}

We set up our comparison.

We work with the latin transcriptions, in order to avoid complications with right-to-left writing in 
the displays of situations where discrepancies occur.

We make pointers to the functions to get the latin transcription for the words of each edition.

To get the latin for word 100 in LK we have to say

```
F[LK].lettersn.v(100)
```

which means that we retrieve the value of feature `lettersn` for node `100` from the feature data of the
LK datasource.

So `F[LK].lettersn.v` and `F[AF].lettersn.v` are the functions we need.
We do not have to implement those functions, they already exist in Text-Fabric, and we store them in 
two global variables for easy reference (also the program runs faster that way).

In [8]:
getTextLK = F[LK].lettersn.v
getTextAF = F[AF].lettersn.v

We also make some further abbreviations for much used values:

In [9]:
maxLK = maxSlot[LK]
maxAF = maxSlot[AF]

# Set up the comparison

We initialize the variables that contain the resulting alignment table.

The table itself is stored in the list `alignment`.
The elements of this list are tuples with a slot number in LK and a corresponding slot number in
AF and a measure of how well these slots correspond with each other.

Some words cannot be matched with the other source. In those cases the corresponding
member of the tuple is the empty string.

That means that the table becomes longer than each of the editions, and that it is not
trivial to locate a specific slot in the alignment list.

For that reason we maintain two indexes that map slot numbers the editions to 
positions in the alignment list.

In [216]:
alignment = []
indexLK = {}
indexAF = {}

We define auxiliary functions for printing the alignment table or portions of it.

In [262]:
def printLines(start=0, end=None):
    """Print specified entries in the alignment table.
    
    Parameters
    ----------
    start: integer, optional 0
        Position of first alignment entry that needs to be printed.
        If it is 0 or negative, it starts from the beginning.
        
    end: integer, optional `None`
        Position of last alignment entry that needs to be printed.
        If it is negative, it indicates a position that much
        before the end of the list.
        If it is absent, it indicates the last line of the list.
    """
    if start < 0:
        start = 0
    if end is None or end > len(alignment):
        end = len(alignment)
    lines = []
    for (iLK, left, d, r, right, iAF) in alignment[start:end]:
        textLK = getTextLK(iLK) if iLK else ""
        textAF = getTextAF(iAF) if iAF else ""
        lines.append(f"{iLK:>5} {left:<2} {textLK:>20} @{d:<2}~{r:3.1f} {textAF:<20} {right:>2} {iAF:>5}")
    return "\n".join(lines)
        
        
def printAlignment(path, asTsv=False):
    """Prints the whole alignment table to file.
    
    Parameters
    ----------
    path: string
        The path name of the file to which the information is printed.
        
    asTsv: boolean, optional `False
        If False, pretty prints the table to file, using 
        `printLines` above.
        If True, write essential data only in tab separated format.
        The essential information is *slot in LK*, *distance*, *slot in AF*
    """
    with open(path, "w") as fh:
        if asTsv:
            for item in alignment:
                fh.write("\t".join(str(it) for it in item) + "\n")
        else:
            fh.write(printLines())
            fh.write("\n")

            
def printDiff(before, after):
    """Print the last alignment entries plus what comes after.
    
    This function is useful if alignment fails at some point.
    It then shows what happened before the failure and how it looks
    after the failure.
    
    Parameters
    ----------
    before: integer
        The amount of alignment entries to print.
        They will be picked from the end of the current alignment table.
    after: integer
        The amount of slots after the last alignment entry that
        needs to be shown for each source.
    """
    print(printLines(start=len(alignment) - before))
    print("^" * 67)
    lastLK = None
    lastAF = None
    for c in range(len(alignment) - 1, -1, -1):
        comp = alignment[c]
        if lastLK is None:
            if comp[0]:
                lastLK = comp[0]
        if lastAF is None:
            if comp[-1]:
                lastAF = comp[-1]
        if lastLK is not None and lastAF is not None:
            break
    if lastLK is not None and lastAF is not None:
        for i in range(after):
            iLK = lastLK + 1 + i
            iAF = lastAF + 1 + i
            textLK = getTextLK(iLK) if iLK <= maxLK else ""
            textAF = getTextAF(iAF) if iAF <= maxAF else ""
            d = distance(textLK, textAF)
            r = ratio(textLK, textAF)
            print(f"{iLK:>5} =  {textLK:>20} @{d:>2}~{r:3.1f} {textAF:<20}  = {iAF:>5}")

# Alignment algorithm

We align the editions by walking through both of them in parallel and performing
comparison actions at each point.

Comparison actions may succeed, in which case we jump in both sources to the first
point after the compared material.

Comparison actions can also fail, in which case we stop the process prematurely
and print the situation around the point of failure.

## Comparison

We do not only compare single words, we also try out short sequences of words at both sides.
We do this because the editions do not always agree on word boundaries.

So, we need to compute whether $n$ consecutive words left are similar to $m$ consecutive words
right.

We set a boundary $C$ on the amount of words that we combine at each source.

We need to try out to al possible combinations, from simplest and shortest to longest and most complex.

Every combination can be characterized by $(n, m)$, where $n$ is the number of words on the left
and $m$ is the number of words on the right. $n$ and $m$ are in the range $1 \ldots C$.

Suppose $C = 3$, then we want to compare combinations in the following order:

combination|x
---|---
$(1, 1)$|--
$(1, 2)$|--
$(2, 1)$|--
$(2, 2)$|--
$(1, 3)$|--
$(3, 1)$|--
$(2, 3)$|--
$(3, 2)$|--
$(3, 3)$|--

In fact, we list all possible combinations and then sort them first by sum of the pair 
and then by decreasing difference of the pair.

We fix a `C` (called `COMBI`) and compute the sequence of combinations up front.

It turns out that 4 is a good value for this source.
3 is definitely too low.
5 is too high, because then the following is likely to happen.

Suppose on the left we have words A, BB, CCC, DDD, EEE and on the right we have BB, CCC, DDD, EEE, X.

The best alignment is to decide that A on the left is not matched with anything on the right, 
X on the left is not matched to anything on the left, and BB, CCC, DDD, EEE will be exact matches.

Buth when comparing combinations of length 5, we cat the comparison

ABBCCCDDDEEE versus BBCCCDDDEEEX with distance and ratio:

In [241]:
left = "ABBCCCDDDEEE"
right = "BBCCCDDDEEEX"

print(f"{distance(left, right)=}")
print(f"{ratio(left, right)=}")

distance(left, right)=2
ratio(left, right)=0.9166666666666666


whereas when we go no further than 4, we would get:

In [242]:
left = "ABBCCCDDD"
right = "BBCCCDDDEEE"

print(f"{distance(left, right)=}")
print(f"{ratio(left, right)=}")

distance(left, right)=4
ratio(left, right)=0.8


The algorithm specifies limits in terms of distance and ratio to determine when a comparison succeeds or fails,
and as you see, the combination of 5 is much more likely to succeed than the combination of 4.

And we want it to fail in this case, because it is a suboptimal match.

In [243]:
COMBINE = 4


def getCombis(c):
    combis = []
    for i in range(1, c + 1):
        for j in range(1, c + 1):
            if i != 1 or j != 1:
                combis.append((i, j))
    return tuple(sorted(combis, key=lambda x: (x[0] + x[1], abs(x[0] - x[1]))))


COMBIS = getCombis(COMBINE)
COMBIS

((1, 2),
 (2, 1),
 (2, 2),
 (1, 3),
 (3, 1),
 (2, 3),
 (3, 2),
 (1, 4),
 (4, 1),
 (3, 3),
 (2, 4),
 (4, 2),
 (3, 4),
 (4, 3),
 (4, 4))

# Single step

Here we define functions that are used to figure out what happens at a single point and that take
actions accordingly.

## Similarity

We compute the similarity of words, and report if the words are similar with respect
to a given level of *strictness*.

The strictness is either an integer or a float.

If it is an integer, it is taken to be an edit distance.
The two words are deemed similar if their edit distance is at most the strictness.

If it is a float, it is taken to be a ratio.
The two words are deemed similar if their ratio is at least the strictness.

The function returns a tuple with a boolean telling whether the two words are similar,
and an integer which is the edit distance.

Note that even if the strictness is passed as a ratio, the result contains the edit distance.

In [244]:
def similar(s1, s2, maxD, minR):
    if s1 == s2:
        return (True, 0, 1.0)
    
    if maxD == 0 or minR == 1:
        return (False, 77, 0.0)
    
    d = distance(s1, s2)
    r = ratio(s1, s2)
    return (d <= maxD and r >= minR, d, r)

## Catching up

In certain cases, there is a gap between comparisons, because one of the sources is missing material.

then we have to fill up the alignment with a number of entries that only contain material
from the one source.

We have two functions to do that, one for LK and one for AF.

Both are passed starting and end positions, which are slot numbers in the corresponding editions.

In the alignment table these entries are flagged by recording a distance of `99`.

In [245]:
def catchupAF(start, end):
    for i in range(start, end + 1):
        indexAF[i] = len(alignment)
        alignment.append(("", 0, 99, 0.0, "", i))
        
        
def catchupLK(start, end):
    for i in range(start, end + 1):
        indexLK[i] = len(alignment)
        alignment.append((i, "", 99, 0.0, 0, ""))

## Special cases

The algorithm can be helped greatly by explicitly telling it what to do at a specific spot.

Do this if an unusual combination of circumstances occurs, for which it is too cumbersome
to spell out code to work around it.

In our case, we need very few special cases, just a handful.

A case is triggered by a specific slot number on the left, and it prescribes that a certain amount of words
should be taken to match a certain other amount of words on the right.

In the alignment table we flag it by recording a distance of `88`.

In [246]:
def doCase(iLK, iAF, debug=False):
    if iLK not in cases:
        return None
    
    (cLK, cAF) = cases[iLK]
    common = min((cLK, cAF))
    for i in range(max((cLK, cAF))):
        nAlignment = len(alignment)
        if i < common:
            alignment.append((iLK + i, cLK, 88, 0.0, cAF, iAF + i))
            indexLK[iLK + i] = nAlignment
            indexAF[iAF + i] = nAlignment
        elif i < cLK:
            alignment.append((iLK + i, cLK, 88, 0.0, cAF, ""))
            indexLK[iLK + i] = nAlignment
        else:
            alignment.append(("", cLK, 88, 0.0, cAF, iAF + i))
            indexAF[iAF + i] = nAlignment
    if debug:
        print(f"[{iLK}~{iAF}] special case ({cLK}, {cAF})")
    return (iLK + cLK, iAF + cAF)

## Combining

We have a function to check out all possible combinations at a certain spot,
and find out whether there is one that meets the given strictness.

If so, the successful match is appended to the alignment table, and the positions
of the next comparison action between both sources is returned.

Otherwise, None is returned.

In [247]:
def findCombi(iLK, iAF, maxD, minR):
    found = None
    
    for (cLK, cAF) in COMBIS:
        if iLK + cLK > maxLK or iAF + cAF > maxLK:
            continue
        textLK = "".join(getTextLK(iLK + i) for i in range(cLK))
        textAF = "".join(getTextAF(iAF + i) for i in range(cAF))
        (isSimilar, d, r) = similar(textLK, textAF, maxD, minR)
        if isSimilar:
            found = (cLK, cAF)
            common = min((cLK, cAF))
            for i in range(max((cLK, cAF))):
                nAlignment = len(alignment)
                if i < common:
                    alignment.append((iLK + i, cLK, d, r, cAF, iAF + i))
                    indexLK[iLK + i] = nAlignment
                    indexAF[iAF + i] = nAlignment
                elif i < cLK:
                    alignment.append((iLK + i, cLK, d, r, cAF, ""))
                    indexLK[iLK + i] = nAlignment
                elif i < cAF:
                    alignment.append(("", cLK, d, r, cAF, iAF + i))
                    indexAF[iAF + i] = nAlignment
            break
    return found

## Comparing

Here we make a local comparison.

First we make a quick 1-1 comparison between the words in both editions at the current positions.
If that fails, we check whether the next word at both sides match.
If that fails, we try out all possible short combinations, using `findCombi` above.

If all fails, None is returned. In case of success, the alignment table is appended to and 
the next positions in both editions are returned.

In [248]:
def compare(iLK, iAF, maxD, minR, debug=False):
    textLK = getTextLK(iLK)
    textAF = getTextAF(iAF)
    (isSimilar, d, r) = similar(textLK, textAF, maxD, minR)
    if isSimilar:
        nAlignment = len(alignment)
        alignment.append((iLK, "", d, r, "", iAF))
        indexLK[iLK] = nAlignment
        indexAF[iAF] = nAlignment
        if debug:
            print(f"[{iLK}~{iAF}] single comparison with distance <= {maxD} and ratio >= {minR}")
        return (iLK + 1, iAF + 1)
    
    if iLK < maxLK and iAF < maxAF:
        textLK = getTextLK(iLK + 1)
        textAF = getTextAF(iAF + 1)
        (isSimilarNext, dNext, rNext) = similar(textLK, textAF, maxD, minR)
        if isSimilarNext:
            nAlignment = len(alignment)
            alignment.append((iLK, "", dNext, rNext, "", iAF))
            indexLK[iLK] = nAlignment
            indexAF[iAF] = nAlignment
            if debug:
                print(f"[{iLK}~{iAF}] single comparison failed with distance <= {maxD} and ratio >= {minR}")
            nAlignment = len(alignment)
            alignment.append((iLK + 1, "", dNext, rNext, "", iAF + 1))
            indexLK[iLK] = nAlignment
            indexAF[iAF] = nAlignment
            if debug:
                print(f"[{iLK}~{iAF}] single comparison recovered with distance <= {maxD} and ratio >= {minR}")
            return (iLK + 2, iAF + 2)

    combi = findCombi(iLK, iAF, maxD, minR)
    if combi is not None:
        (cLK, cAF) = combi
        if debug:
            print(f"[{iLK}~{iAF}] ({cLK}, {cAF}) comparison with distance <= {maxD} and ratio >= {minR}")
        return (iLK + cLK, iAF + cAF)
    
    return None

## Looking up

If comparison fails, even after having tried all possible combinations of words, we are potentially stuck.

But before giving up, we can make a jump if we can match the current word left to a word right that is not far ahead.
Or vice versa.

The following function checks whether there are successful jumps within a specific region and within a given strictness.
Typically, this function is called with different strictnesses for different regions.
For example, first we want to try short jumps with moderate strictness, and if that fails, we try out longer
jumps with higher strictness.

Short jumps are tried out before long jumps, and we try the jumps in the left and right edition alternately.

If a jump is successful, the next position will be returned, and a `catchup` will be done for the
edition in which the jump has been made.
When there is no successful jump, None is returned.

The catchup is a bit complicated, because the comparison that led to the successful jump has already been appended
to the alignment table. So we have to pop that first (it could be multiple entries in case the comparison involved
a combination of words), and then we can do the catchup, and then we can push the comparison entries again.
We also need to update the indexes between slot numbers and the alignment table entries.

In [249]:
def lookup(iLK, iAF, maxD, minR, start, end, debug=False):
    step = None
    
    for i in range(start, end + 1):
        prevAlignmentIndex = len(alignment)
        
        if iAF + i <= maxAF:
            step = compare(iLK, iAF + i, maxD, minR, debug=debug)
            if step:
                if debug:
                    print(f"[{iLK}~{iAF}] right {i}-jump to {iAF + i}")
                thisAlignment = list(alignment[prevAlignmentIndex:])
                alignment[prevAlignmentIndex:] = []
                
                catchupAF(iAF, iAF + i - 1)
                for thisComp in thisAlignment:
                    nAlignment = len(alignment)
                    thisLK = thisComp[0]
                    thisAF = thisComp[-1]
                    if thisLK:
                        indexLK[thisLK] = nAlignment
                    if thisAF:
                        indexAF[thisAF] = nAlignment
                    alignment.append(thisComp)
                break

        if iLK + i <= maxLK:
            step = compare(iLK + i, iAF, maxD, minR, debug=debug)
            if step:
                if debug:
                    print(f"[{iLK}~{iAF}] left {i}-jump to {iLK + i}")
                thisAlignment = list(alignment[prevAlignmentIndex:])
                alignment[prevAlignmentIndex:] = []
                
                catchupLK(iLK, iLK + i -1)
                for thisComp in thisAlignment:
                    nAlignment = len(alignment)
                    thisLK = thisComp[0]
                    thisAF = thisComp[-1]
                    if thisLK:
                        indexLK[thisLK] = nAlignment
                    if thisAF:
                        indexAF[thisAF] = nAlignment
                    alignment.append(thisComp)
                break
    return step

# Orchestrating

We can now define how we are going to orchestrate the comparisons that will build the alignment table.

First we define a function that does a single step or a limited number of steps,
starting at given positions in the editions.

This function will then be called from a big loop.

The comparison consists of checking out a number of things under several strictness conditions.
As soon as a check succeeds, we perform the actions associated with that, and continue to the next iteration.

1. First of all: is there a special case defined for this point? If so, follow its instructions.
2. Do left and right compare as equal, give or take an edit distance of 1?
   Comparing involves not only comparing the single words at the current positions,
   but also trying out a number of word combinations at the current point.
3. Do left and right compare as equal, give or take an edit distance of 2?
4. Do left and right compare as equal, give or take an edit distance of 3?
5. Do left and right compare as equal, give or take a similarity ratio of 0.5?
   This is rather lenient for long words.
6. If all comparisons have failed so far, it is time to jump forward:
7. is there a match within 5 words, give or take a similarity ratio of 0.6?
   This is rather coarse, but we stay in the neighbourhood.
8. Is there a match between 6 and 10 words, give or take a similarity ratio of 0.8?
   We are drifting further from home, so we pose stricter requirements to the match.
9. Is there an exact match between and 1000 words?
   When looking so far forward, we do not trust matches that are not 100%.
   
Clearly, there is no guarantee that any of these will be successfull.
When it fails, we have to inspect the failure and see what happens there.
There are basically three solutions to overcome such roadblocks

1. Define a special case. Easy. Until it appears that too many special cases are needed.
2. Tweak the strictness levels in the algorithm. Easy, but risky. Things that went well
   may now go awry. It is not only about preventing the roadblocks, but also maintaining
   the quality of the output alignment table.
3. Change the orchestration: change the order of comparing and jumping, introduce more strictness levels,
   try more or less combinations. Invent new criteria for comparison.
   This is really difficult. The present orchestration is the result of quite some trial and error.

In [270]:
decisions = collections.Counter()


def doDiffs(startLK=1, startAF=1, steps=-1, show=False, debug=False):
    alignment.clear()
    decisions.clear()
    
    lastCase = None
    step = (startLK, startAF)

    complete = False
    it = 0
    
    def doDiff(iLK, iAF):
            decision = 0
            if iLK > maxLK or iAF > maxAF:
                if iAF < maxAF:
                    catchupAF(iAF, maxAF)
                if iLK < maxLK:
                    catchupLK(iLK, maxLK)
                decisions[decision] += 1
                return True

            nonlocal lastCase

            decision += 1

            if lastCase != iLK:
                step = doCase(iLK, iAF, debug=debug)
                if step:
                    lastCase = iLK
                    decisions[decision] += 1
                    return step

            step = compare(iLK, iAF, 0, 1.0, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = compare(iLK, iAF, 1, 0.8, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 0, 1.0, 1, 5, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = compare(iLK, iAF, 1, 0.8, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = compare(iLK, iAF, 2, 0.7, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = compare(iLK, iAF, 3, 0.6, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 0, 1.0, 1, 10, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 1, 0.8, 1, 10, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 0, 1.0, 10, 20, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 1, 0.8, 10, 20, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 2, 0.7, 1, 20, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 1, 0.8, 20, 100, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            step = lookup(iLK, iAF, 2, 0.7, 20, 100, debug=debug)
            decision += 1
            if step:
                decisions[decision] += 1
                return step

            return False
        
    while it != steps:
        it += 1
        step = doDiff(*step)
        
        if step is True:
            printAlignment("zipLK-AF-complete.txt")
            printAlignment(ALIGNMENT_PATH, asTsv=True)
            print(f"Alignment complete, {len(alignment)} entries.")
            break
        elif step is False:
            printAlignment("zipLK-AF-incomplete.txt")
            print(f"Alignment blocked, {len(alignment)} entries.")
            printDiff(20, 20)
            break
            
    if show:
        print(printLines())
        
    for d in sorted(decisions):
        n = decisions[d]
        print(f"{d:>2} taken: {n:>5} x")


# The diff function

Finally we can write up the `doDiff` function which loops through the editions and produces an alignment table.

But you can also run call it for a specific pair of positions and a limited number of steps.
Handy for debugging and tweaking the decision parameters.

You specify the start positions by means of the slot numbers in the LK and AF.
You can specify the number of steps, or pass -1 in order to continue to the end.

And if you pass `show=True`, the alignment table will be printed after completion.
Only do this if you run a limited number of steps.

# Check the result

We must make sure that the algorithm has not skipped material, duplicated material, or put material in the wrong order.

We examine the alignment list and check that we have all slot numbers of LK in the right order and all slot numbers of AF idem.

The mapping itself is needed elsewhere in Text-Fabric, let us write it to file.
We write it as an edge feature into the AF edition.

In [272]:
# this number of good lines between bad lines will not lead to the
# interruption of bad stretches

LOOKAHEAD = 3


def analyseStretch(start, end):
    total = 0
    good = 0
    onlyLK = 0
    onlyAF = 0
    
    for (iLK, left, d, r, right, iAF) in alignment[start:end + 1]:
        total += 1
        if not iLK:
            onlyAF += 1
        if not iAF:
            onlyLK += 1
        if d == 0:
            good += 1
    
    suspect = onlyAF > 1 and onlyLK > 1 and onlyAF + onlyLK > 5
    return suspect
    
def checkAlignment():
    errors = {}
    prevILK = 0
    prevIAF = 0
    
    where = collections.Counter()
    agreement = collections.Counter()
    badStretches = collections.defaultdict(lambda: [])
    combinations = collections.defaultdict(lambda: [])
    
    startBad = 0
    startCombi = 0
    nCombi = 0
    
    for (c, (iLK, left, d, r, right, iAF)) in enumerate(alignment):
        thisBad = d > 0 or not iLK or not iAF
        hasLeft = left != ""
        hasRight = right != ""
        
        # a good line between bad lines is counted as bad
        if not thisBad and startBad:
            nextGood = True
            for j in range(1, LOOKAHEAD + 1):
                if c + j < len(alignment):
                    compJ = alignment[c + j]
                    if compJ[2] > 0 or not compJ[0] or not compJ[-1]:
                        nextGood = False
                        break
            if not nextGood:
                thisBad = True
        if startBad:
            if not thisBad:
                badStretches[c - startBad].append(startBad)
                startBad = 0
        else:
            if thisBad:
                startBad = c
                
        if startCombi and nCombi == c - startCombi:
            startCombi = 0
            nCombi = 0
        if startCombi == 0:
            thisCombi = hasLeft and left > 1 or hasRight and right > 1
            if thisCombi:
                combinations[(left, right)].append(c)
                startCombi = c
                nCombi = max((left, right))
        
        agreement[d] += 1
        
        if iLK:
            if iLK != prevILK + 1:
                errors.setdefault("wrong iLK", []).append(f"{c:>5}: Expected {prevILK + 1}, found {iLK}")
            prevILK = iLK
            if iAF:
                where["both"] += 1
        else:
            where[AF] += 1
        if iAF:
            if iAF != prevIAF + 1:
                errors.setdefault("wrong iAF", []).append(f"{c:>5}: Expected {prevIAF + 1}, found {iAF}")
            prevIAF = iAF
        else:
            where[LK] += 1
            
    if startBad:
        badStretches[len(alignment) - startBad].append(startBad)
            
    if prevILK < maxLK:
        errors.setdefault("missing iLKs at the end", []).append(f"last is {prevILK}, expected {maxLK}")
    elif prevILK > maxLK:
        errors.setdefault("too many iLKs at the end", []).append(f"last is {prevILK}, expected {maxLK}")
    if prevIAF < maxAF:
        errors.setdefault("missing iAFs at the end", []).append(f"last is {prevIAF}, expected {maxAF}")
    elif prevIAF > maxAF:
        errors.setdefault("too many iAFs at the end", []).append(f"last is {prevIAF}, expected {maxAF}")
    
    print("\nSANITY\n")
    if not errors:
        print("All OK")
    else:
        for (kind, msgs) in errors.items():
            print(f"ERROR {kind} ({len(msgs):>5}x):")
            for msg in msgs[0:10]:
                print(f"\t{msg}")
            if len(msgs) > 10:
                print(f"\t ... and {len(msgs) - 10} more ...")
                
    print(f"\nAGREEMENT\n")
    print("Where are the words?\n")
    print(f"\t{LK}-only: {where[LK]:>5} slots")
    print(f"\t{AF}-only: {where[AF]:>5} slots")
    print(f"\tboth:    {where['both']:>5} slots")
    
    print("\nHow well is the agreement?\n")
    for (d, n) in sorted(agreement.items()):
        print(f"edit distance {d:>3} : {n:>5} words")
    print(f"NB: 88 are special cases that have been declared explicitly")
    
    print(f"\nCOMBINATIONS\n")
    print(f"What combination alignments are there and how many?")
    for comb in sorted(combinations):
        cs = combinations[comb]
        (left, right) = comb
        print(f"\t({left:>2}, {right:>2}) : {len(cs):>4} x :")
        for (i, c) in enumerate(cs[0:3]):
            print(f"EXAMPLE {i + 1}:")
            print(printLines(start=max((1, c - 2)), end=min((len(alignment), c + 2 + max(comb)))))
            print("")
    
    print(f"\nBAD STRETCHES\n")
    print("How many of which size?\n")
    allSuspects = []
    someBenigns = []
    for (size, starts) in sorted(badStretches.items(), key=lambda x: (-x[0], x[1])):
        suspects = {start: size for start in starts if analyseStretch(start, start + size)}
        benigns = {start: size for start in starts if start not in suspects}
        allSuspects.extend([(start, start + size) for (start, size) in suspects.items()])
        someBenigns.extend([(start, start + size) for (start, size) in list(benigns.items())[0:3]])
        examples = ", ".join(str(start) for start in list(suspects.keys())[0:3])
        if not suspects:
            examples = ", ".join(str(start) for start in list(benigns.keys())[0:3])
        print(f"bad stretches of size {size:>3} : {len(suspects):>4} suspect of total {len(starts):>4} x see e.g. {examples}")
        
    print(f"\nShowing all {len(allSuspects)} inversion suspects" if len(allSuspects) else "\nNo suspect bad stretches\n")
    for (i, (start, end)) in enumerate(reversed(allSuspects)):
        print(f"\nSUSPECT {i + 1:>2}")
        print(printLines(max((1, start - 5)), min((len(alignment), end + 5))))
    print(f"\nShowing some ({len(someBenigns)}) benign examples" if len(someBenigns) else "\nNo bad stretches\n")
    for (i, (start, end)) in enumerate(someBenigns):
        print(f"\nBENIGN {i + 1:>2}")
        print(printLines(max((1, start - 2)), min((len(alignment), end + 2))))

# Define the special cases

The special cases below are the result of trial and error.

The keys are the LK slot numbers where the cases must be applied.

The values are the amount of words in LK and in AF that will bee identified.

So `1000: (3, 4)` means that at slot position 1000 in LK we take 3 consecutive words and identify it with
4 consecutive words in AF at the current position there.

When you read the alignment table (by `printDiff` or `printLines`)
you see the information on the basis of which you could define a special case.

In [290]:
cases = {
    3072: (5, 1),
    4597: (1, 1),
    4598: (1, 1),
    4600: (0, 4),
    8273: (4, 0),
    13539: (2, 1),
    14878: (1, 1),
    14879: (1, 0),
    14880: (12, 0),
    16198: (1, 1),
    16199: (1, 1),
    16200: (1, 1),
    16201: (1, 1),
    16212: (1, 1),
    18029: (6, 1),
    22762: (1, 0),
    22763: (1, 1),
    22764: (1, 1),
}

# Run the comparison

Here we go!

In [292]:
doDiffs()

Alignment complete, 40983 entries.
 0 taken:     1 x
 1 taken:    18 x
 2 taken: 36340 x
 3 taken:   538 x
 4 taken:   312 x
 6 taken:    32 x
 7 taken:    12 x
 8 taken:     1 x
 9 taken:     9 x
12 taken:     1 x


# Check the results

The result of the alignment is a table where the first column contains all the slots in LK,
and the second column all the slots in AF:

Left you see the LK slot numbers and the words in those slots.

Right you see the AF words and their AF slot numbers.

The middle column is a measure for the edit distance between the words at both sides.

`99` means: at one of the sides the word is missing.
`88` means: this was a prescribed, special case, no edit distance has been measured.

All other values are the number of edits you have to make in order to change one word to the other.

Sometimes words are combined: a number of words left corresponds to a posiibly different number of words right.
You see that indicated by `+n` and `n+`. If the numbers are not equal, empty words are inserted with `^n` or `n^`.

If just a single word left is aligned with a single word right, you see it marked with `=` on the left and right.

In [293]:
print(printLines(start=0, end=20))

      0                       @99~0.0 bnzlylālʿ                   1
      0                       @99~0.0 ylrʿā                       2
    1                   ālḥmd @0 ~1.0 ālḥmd                       3
    2                     llh @0 ~1.0 lh                          4
    3                    mnzl @0 ~1.0 mnzl                        5
    4                   ālḥkm @0 ~1.0 ālḥk                        6
    5                     ʿlá @0 ~1.0 ʿlá                         7
    6                    ḳlwb @0 ~1.0 ḳlwb                        8
    7                   ālklm @0 ~1.0 ālklm                       9
    8                  bāḥdyŧ @0 ~1.0 bāḥdyŧ                     10
    9                  ālṭryḳ @0 ~1.0 ālṭryḳ                     11
   10                   ālāmm @0 ~1.0 ālāmm                      12
   11                      mn @0 ~1.0 mn                         13
   12                  ālmḳām @0 ~1.0 ālmḳām                     14
   13 1                ālāḳdm @1 ~0.9 ā         

# Quality check

How did the alignment perform?
It did complete, but what have we got?
It could be just garbage.

## Sanity

First of all we need to know whether all words of both LF and LK occur left and right, without gaps and duplications
and in the right order. We check that.

## Agreement

We provide information on about the agreement in both sources.
How many words are there for which there is no alignment inb the other edition?

And how close are the words for which an alignment could be established?

Note that there are two reasons for bad agreement results:

1. The editions are really very different
2. The alignment is not optimal and fails to align many words that would have matched under another
   alignment strategy.
   
## Bad stretches

Are there long stretches of poorly matching alignments?
We are going to examine them.

If they contain many cases of a left missing word and many cases of a right missing word,
they are suspect, because they might contain largely the same words, but the algorithm has failed
to spot them.

We show all suspect bad stretches.

The remaining stretches are benign.
We show examples of benign bad strectches (at most three examples per size).

In [294]:
checkAlignment()


SANITY

All OK

AGREEMENT

Where are the words?

	LK-only:   712 slots
	AF-only:   604 slots
	both:    39667 slots

How well is the agreement?

edit distance   0 : 39507 words
edit distance   1 :   908 words
edit distance   2 :    51 words
edit distance   3 :    11 words
edit distance  88 :    45 words
edit distance  99 :   461 words
NB: 88 are special cases that have been declared explicitly

COMBINATIONS

What combination alignments are there and how many?
	( 0,  4) :    1 x :
EXAMPLE 1:
 4598 1                 nwḥyŧ @88~0.0 wḥh                   1  4555
 4599                    āʿlm @0 ~1.0 āʿlm                     4556
      0                       @88~0.0 āydlk                 4  4557
      0                       @88~0.0 āllh                  4  4558
      0                       @88~0.0 brwḥ                  4  4559
      0                       @88~0.0 mnh                   4  4560
 4600                      ān @0 ~1.0 ān                       4561
 4601                 āltnzy

# Test cases

In [193]:
doDiffs(startLK=178, startAF=182, steps=4, show=True, debug=True)

[178~182] single comparison with strictness 0
[180~183] single comparison with strictness 0 failed
[180~183] single comparison with strictness 0 recovered
[179~183] left 1-jump to 180
[182~185] single comparison with strictness 1
[183~186] single comparison with strictness 0 failed
[183~186] single comparison with strictness 0 recovered
  178                    ālḥḳ @0  ālḥḳ                      182
  179                   tʿālá @99                       0      
  180                     lmā @0  mā                        183
  181                     smʿ @0  smʿ                       184
  182                   dʿāʾy @1  dʿāy                      185
  183                      ḳd @0  fd                        186
  184                    āǧāb @0  āǧāb                      187
 2 taken:     2 x
 3 taken:     1 x
 4 taken:     1 x


In [194]:
doDiffs(startLK=13347, startAF=13338, steps=4, show=True, debug=True)

[13347~13338] single comparison with strictness 0
[13348~13339] single comparison with strictness 0
[13349~13340] (2, 2) comparison with strictness 0
[13351~13342] single comparison with strictness 0
13347                  ālālhy @0  ālālhy                  13338
13348                     flā @0  flā                     13339
13349 2                   ḳrb @0  ḳrbā                  2 13340
13350 2                  āḳrb @0  ḳrb                   2 13341
13351                      mn @0  mn                      13342
 2 taken:     4 x


In [195]:
doDiffs(startLK=5635, startAF=5607, steps=4, show=True, debug=True)

[5635~5607] single comparison with strictness 3 failed
[5635~5607] single comparison with strictness 3 recovered
[5637~5618] single comparison with strictness 0
[5637~5609] right 9-jump to 5618
[5638~5619] single comparison with strictness 0
[5639~5620] single comparison with strictness 0
 5635                     ḳyl @3  smwhm                    5607
 5636                     lhm @3  flw                      5608
      0                       @99 smwhm                    5609
      0                       @99 ā                        5610
      0                       @99 lsmwhm                   5611
      0                       @99 ḥǧārŧ                    5612
      0                       @99 wšǧrā                    5613
      0                       @99 wkwkbā                   5614
      0                       @99 wlw                      5615
      0                       @99 ḳyl                      5616
      0                       @99 lhm                      5617
 5637 

In [196]:
doDiffs(startLK=11794, startAF=11799, steps=4, show=True, debug=True)

[11794~11799] single comparison with strictness 0 failed
[11794~11799] single comparison with strictness 0 recovered
[11796~11801] single comparison with strictness 0
[11797~11802] single comparison with strictness 0
[11798~11803] single comparison with strictness 0
11794                   ānḳṣt @0  tḳf                     11799
11795                    ʿlyh @0  ʿlyh                    11800
11796                      ān @0  ān                      11801
11797                     šāʾ @0  šāʾ                     11802
11798                    āllh @0  āllh                    11803
 2 taken:     4 x


In [234]:
doDiffs(startLK=27434, startAF=27337, steps=2, show=True, debug=True)

[27434~27337] single comparison with strictness 1
[27442~27338] single comparison with strictness 1
[27435~27338] left 7-jump to 27442
27434                     kmā @1 ~0.8 kā                      27337
27435                   nsbth @99~0.0                       0      
27436                 ālfwḳyŧ @99~0.0                       0      
27437                    ālyh @99~0.0                       0      
27438                      fy @99~0.0                       0      
27439                    ḳwlh @99~0.0                       0      
27440                  yḫāfwn @99~0.0                       0      
27441                    rbhm @99~0.0                       0      
27442                      mn @1 ~0.5 ān                      27338
 3 taken:     1 x
 9 taken:     1 x


In [277]:
doDiffs(startLK=4595, startAF=4552, steps=10, show=True, debug=True)

[4595~4552] single comparison with distance <= 1 and ratio >= 0.8
[4596~4553] single comparison with distance <= 3 and ratio >= 0.6
[4597~4554] special case (1, 1)
[4598~4555] special case (1, 1)
[4599~4556] single comparison with distance <= 0 and ratio >= 1.0
[4600~4557] special case (0, 4)
[4600~4561] single comparison with distance <= 0 and ratio >= 1.0
[4601~4562] single comparison with distance <= 0 and ratio >= 1.0
[4602~4563] single comparison with distance <= 0 and ratio >= 1.0
[4603~4564] single comparison with distance <= 0 and ratio >= 1.0
 4595                  sbwḥyŧ @1 ~0.8 sbwḥyh                   4552
 4596                      fy @1 ~0.7 y                        4553
 4597 1                  klmŧ @88~0.0 lʿh                   1  4554
 4598 1                 nwḥyŧ @88~0.0 wḥh                   1  4555
 4599                    āʿlm @0 ~1.0 āʿlm                     4556
      0                       @88~0.0 āydlk                 4  4557
      0                       @88~

In [286]:
doDiffs(startLK=16197, startAF=16133, steps=10, show=True, debug=True)

[16197~16133] single comparison with distance <= 0 and ratio >= 1.0
[16198~16134] special case (1, 1)
[16199~16135] special case (1, 1)
[16200~16136] special case (1, 1)
[16201~16137] special case (1, 1)
[16202~16138] single comparison with distance <= 0 and ratio >= 1.0
[16203~16139] single comparison with distance <= 0 and ratio >= 1.0
[16204~16140] single comparison with distance <= 0 and ratio >= 1.0
[16205~16141] single comparison with distance <= 0 and ratio >= 1.0
[16206~16142] single comparison with distance <= 0 and ratio >= 1.0
16197                      fy @0 ~1.0 fy                      16133
16198 1              ālʿārfyn @88~0.0 ālʿārf                1 16134
16199 1                   tḳf @88~0.0 yḳf                   1 16135
16200 1                 ʿndhā @88~0.0 ʿndhā                 1 16136
16201 1                    bl @88~0.0 l                     1 16137
16202                      hw @0 ~1.0 hw                      16138
16203                  ālʿārf @0 ~1.0 ālʿārf    

In [291]:
doDiffs(startLK=16210, startAF=16146, steps=15, show=True, debug=True)

[16210~16146] single comparison with distance <= 0 and ratio >= 1.0
[16211~16147] single comparison with distance <= 0 and ratio >= 1.0
[16212~16148] special case (1, 1)
[16213~16149] single comparison failed with distance <= 0 and ratio >= 1.0
[16213~16149] single comparison recovered with distance <= 0 and ratio >= 1.0
[16215~16151] single comparison with distance <= 0 and ratio >= 1.0
[16216~16152] (2, 3) comparison with distance <= 1 and ratio >= 0.8
[16218~16155] (2, 3) comparison with distance <= 2 and ratio >= 0.7
[16220~16158] single comparison with distance <= 0 and ratio >= 1.0
[16221~16159] single comparison with distance <= 0 and ratio >= 1.0
[16222~16160] (2, 1) comparison with distance <= 0 and ratio >= 1.0
[16224~16161] single comparison with distance <= 0 and ratio >= 1.0
[16225~16162] single comparison with distance <= 0 and ratio >= 1.0
[16226~16163] single comparison with distance <= 0 and ratio >= 1.0
[16227~16164] (2, 1) comparison with distance <= 0 and ratio >= 1